In [ ]:
import numpy as np
from numpy import vectorize

import matplotlib.pyplot as plt
%matplotlib inline

from rtbm import RTBM, minimizer
from rtbm.costfunctions import logarithmic

## Generate data

In [ ]:
# likas 1d example uniform
n = 5000

data = np.random.uniform(-5,5,n)
data = data.reshape(1,n)
plt.hist(data.T, normed=True, bins=50);

In [ ]:
# likas 1d example
n = 5000

data = np.zeros(n)
u = np.random.random_sample(n)
for i in range(n):
    if u[i] < 0.25:
        data[i] = np.random.normal(-7, 0.5)
    elif u[i] < 0.50:
        data[i] = np.random.uniform(-3,-1)
    elif u[i] < 0.75:
        data[i] = np.random.uniform(1,3)
    else:
        data[i] = np.random.normal(7, 0.5)
data = data.reshape(1,n)
plt.hist(data.T, normed=True, bins=50);

In [ ]:
# likas 1d example 2
def p(x):
    a = 1/6.5523
    if x < 0 and x >= 3 + 2**0.5:
        return 0
    elif 0 <= x < 2:
        return a*(2-x/2)
    else:
        return a*(2-(x-3)**2)

def rejection_sampler(p,xbounds,pmax):
    while True:
        x = np.random.rand(1)*(xbounds[1]-xbounds[0])+xbounds[0]
        y = np.random.rand(1)*pmax
        if y <= p(x):
            return x

n = 5000
data = np.zeros(n)
for i in range(n):
    data[i] = rejection_sampler(p,[0,3+2**0.5],0.3)
data = data.reshape(1,n)
plt.hist(data.T, normed=True, bins=50);

In [ ]:
# likas example 3, 2d uniform [0,0.2]
n = 5000
data = np.random.uniform(0,0.2,(2,n/2))
plt.figure(figsize=(5,5))
plt.hist2d(data[0,:], data[1,:], bins=50, normed=True);

## Training

In [ ]:
a = RTBM(1,2, init_max_param_bound=20)

In [ ]:
minim = minimizer.CMA(False)
solution = minim.train(logarithmic(), a, data, tolfun=1e-5)

## Testing output

In [ ]:
test_data = (np.linspace(-10,10, 100)).reshape(1, 100)
plt.hist(data.T, bins=50, normed=True);
plt.plot(test_data.flatten(), a(test_data).flatten(), 'o-')

# Testing model

In [ ]:
from rtbm.layers import ThetaUnitLayer, NormAddLayer
from rtbm.model import Model

In [ ]:
mdl = Model()
mdl.add(ThetaUnitLayer(1,2,Nhidden=2))
mdl.add(NormAddLayer(2,1))

In [ ]:
minim = minimizer.CMA(False)
solution = minim.train(logarithmic(), mdl, data, tolfun=1e-5)

In [ ]:
test_data = (np.linspace(-10,10, 100)).reshape(1, 100)
plt.hist(data.T, bins=50, normed=True);
plt.plot(test_data.flatten(), mdl(test_data).flatten(), 'o-')